In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# let's start by importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline 

In [ ]:
plt.style.use('ggplot')

## Importing the Data

In [ ]:
application_df = pd.read_csv(r"C:\Users\Arindham Krishna\OneDrive\Desktop\Projects for Portfolio\Exploratory Data Analysis\Loan Applications Data\application_data.csv\application_data.csv")
prev_ap_df = pd.read_csv(r"C:\Users\Arindham Krishna\OneDrive\Desktop\Projects for Portfolio\Exploratory Data Analysis\Loan Applications Data\previous_application.csv\previous_application.csv")

In [ ]:
application_df.head()

In [ ]:
prev_ap_df.head()

In [ ]:
application_df.info(verbose=True)

In [ ]:
prev_ap_df.info()

We can see that when we use .info(), it will give us the detailed information about the dataset. 
For example, application_df.info() gives an output where we can see the number of records it contains and the number of features it has. 
Application data set has 307511 entries and 122 columns. Again, you can also check the amount of memory the dataset is cosuming. 
Application dataset is consuming 286.2+ MB and the same above line you can see data types and their counts. 

Same goes for the previous application dataset, prev_ap_df.info()

In [ ]:
# Another way to check the number of columns a dataset has, is to go ahead with the .shape command. 
print("shape of application dataset",application_df.shape)
print("shape of prev application dataset",prev_ap_df.shape)

Now, as we know that we need to check the current defaulters from the loan application and also we need to analyse what are the chances that a prospective client might turn out to be a defaulter so that we can avoid the losses. 

Let's first look at how current defaulter to creditor count looks like from the application dataset. 

In [ ]:
plt.title("Payment Status of Current Application")
sns.countplot(application_df['TARGET'])
plt.show()

(1 = Defaulter, 0 = Creditor)From the above countplot, we can see that the defaulter count is comparitively very minimal to the creditors count. 
This defines that our dataset is an imbalaned data.

## What is Imbalanced Data? 

So, while doing any analysis or majorly to do any prediction we always have a target feature. Depending on that target feature we will be able to make the predictions. Now, lets say if the target feature is imbalanced like in this case then the algorithms or classifiers will only pick up the majority values and the minority values will be ignored. In our case, if we give the same data set to any classifier then there are chances that defaulters records will be ignored and only creditors records will be considered and doing so will give us inaccurate predictions. 


Another example other than this is if we consider disease prediction data and in there if 95 patiens are without disease and 5 are with disease then chances are more that the classifier will ignore the minority records. 

#### Checking the ratio

In [ ]:
creditor = application_df[application_df['TARGET']==0]
defaulter = application_df[application_df["TARGET"]==1]

print("Number of Creditors", creditor.shape[0])
print("Number of Defaulters", defaulter.shape[0])

Here, we can see 282,686 (two hundred eighty thousand applications have paid their installments timely)
24,825(twenty five thousand applications are defaulters)

In [ ]:
#Lets chcek the percentage of defaulters
print("Percentage of defaulters:", round(defaulter.shape[0]*100/(creditor.shape[0]+defaulter.shape[0]),2))

Almost 8 percent of applications are into the defualters list. Rest 92 percent are creditors. 

Defaulter : Creditor = 8:92

#### Now lets start with our analysis and find out which clients can be a defaulter and which clients are good prospects and not to loose them by not providing loan.  

Let's start with analysing all the features we have got. 


In [ ]:
#This will print out all the features, in list. We can also use just list(application_df.columns)
print(list(application_df.columns))

### Incestigation 1: Documents and its Impact on Target

In [ ]:
application_documents_df = application_df[['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 
                                           'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 
                                           'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
                                           'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 
                                           'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',]]


In [ ]:
application_documents_df.head()

In [ ]:
plt.figure(figsize=(10,6))
sns.displot(
    data=application_documents_df.melt(value_name="Documents Submitted"),
    y="variable",
    hue="Documents Submitted",
    multiple="fill",
    aspect=1.25
)
plt.show()


(0 = Not Submitted, 1 = Submitted) From the above plot, can see that most of the applications have not submitted all the documents except the Document_3

It's obvious that if these documents were not submitted then they will not make any imapct on our Target. 

However, we will check the correlation between document_3 and Target. 

### Correlation Matrix Between Document 3 and Target

In [ ]:
corr = application_df[["TARGET","FLAG_DOCUMENT_3"]].corr()
corr.style.background_gradient(cmap='coolwarm')

As we know the correlation values range between -1 to 1 and any values nearer or equal to -1 determines a negative correlation, any value nearer to 0 determines no correlation and any value near to 1 or equal to 1 determines that there is correlation. 

From the above matrix, we can see that the correlation values are nearer to 0  and hence document_3 submission does not impact the target value by any chance. We can also go ahead and drop document_3 feature along with other documents. 

## Investigation 2: Clients House Details vs Target 

Lets see if the information provided to us about the size and other details of clients stay has by any chance impact on the target columns. 

It's convenient to print .columns() as you can copy the column names easily. 

In [ ]:
application_houseinfo_df = application_df[['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 
                                           'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 
                                           'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 
                                           'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 
                                           'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 
                                           'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 
                                           'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 
                                           'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 
                                           'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 
                                           'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 
                                           'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
                                           'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 
                                           'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE',
                                           'EMERGENCYSTATE_MODE']]

In [ ]:
application_houseinfo_df.head()

Looks like these features can contain null records, lets investigate for that. 

In [ ]:
print(application_houseinfo_df.isna().sum())
#APARTMENTS_AVG has 156,061 missing records and on a glance, we can see that almost all these features has high missing values.

For more better understanding lets calculate missing values percentage in these particualr 47 columns. 

In [ ]:
# print(round((application_houseinfo_df.isnull().sum()*100/application_df.shape[0]),2))
#At a glance looks like all columns have more than 45% of missing values. Let's sort them to get aware about the range. 
houseinfo_missingdata = round((application_houseinfo_df.isnull().sum()*100/application_df.shape[0]),2)
print(houseinfo_missingdata.sort_values())

#As said that after sorting we can figure the range and here we can see that (47-70%) of data is missing. 
#Hence, its wise to drop these records because we have records of around three hundred thousand and we will still be left be 
# fair amount of records to do the analysis. 

In [ ]:
#Dropping the redundant features. 
application_df.drop(['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 
                                           'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 
                                           'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 
                                           'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 
                                           'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 
                                           'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 
                                           'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 
                                           'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 
                                           'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 
                                           'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 
                                           'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI',
                                           'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI', 
                                           'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'TOTALAREA_MODE', 'WALLSMATERIAL_MODE',
                                           'EMERGENCYSTATE_MODE','FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 
                                           'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 
                                           'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13',
                                           'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 
                                           'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',], axis=1)

The application data had 122 columns initially but after finding out no correlation of particular columns with target we have 
dropped them and now we are left with 307511 rows × 55 columns

### House Type vs Target

In [ ]:
plt.figure()
sns.countplot(application_df["NAME_HOUSING_TYPE"], hue=application_df["TARGET"])
plt.xticks(rotation=90)
plt.tight_layout()
plt.title("Housing Type vs. Target")
plt.show()

In [ ]:
#### Defining a function so that we get percentage of defaulters for that particular column. #####
# Function_name : value_wise_defaulter_percentage
# Usage : Returns % of defaulters for every unique value of a column(Categorical)
# Arguments : dataframe, column
# Returns : a dataframe containing unique values of a caterory and % of defaulters

def value_wise_defaulter_percentage(df, col):
    new_df = pd.DataFrame(columns=['Value', 'Percentage of Defaulter'])
    
    for value in df[col].unique():
        default_cnt = df[(df[col] == value) & (df.TARGET == 1)].shape[0]
        total_cnt = df[df[col] == value].shape[0]
        new_df = new_df.append({'Value' : value , 'Percentage of Defaulter' : (default_cnt*100/total_cnt)}, ignore_index=True)
    return new_df.sort_values(by='Percentage of Defaulter', ascending=False)

In [ ]:
value_wise_defaulter_percentage(application_df, "NAME_HOUSING_TYPE")

It can be seent that clients living in Rented apartment or living with parents have higher chances of being a defaulter.

### Marital Status vs Target

In [ ]:
value_wise_defaulter_percentage(application_df,"NAME_FAMILY_STATUS")

We can see that applicants with Civil Marriage and Signle Staus have higher possibility of being a defaulter. 

### Income Type vs Target and Education Type vs Target

In [ ]:
fig = px.histogram(application_df, x="NAME_INCOME_TYPE", color="TARGET",title= "Income Type vs Target",barmode='group')
fig.update_xaxes(tickangle = -90)

In [ ]:
value_wise_defaulter_percentage(application_df,"NAME_INCOME_TYPE")

Observation: 

Applicants in their Maternity Leave and Applicants who are unemployed have very high chance that they can be defaulter.
It should be avoided or cross checked with other parameters before sanctioning the loan.  

In [ ]:
fig = px.histogram(application_df, x="NAME_EDUCATION_TYPE", color="TARGET",title= "Education Type vs Target",barmode='group')
fig.update_xaxes(tickangle = -90)

In [ ]:
value_wise_defaulter_percentage(application_df,"NAME_EDUCATION_TYPE")

Observation:

Applicants with not proper education background can have the chances of not repaying the loan. Verify the education background before sanctioning the loan. 

## Gender, Age, Income vs Target

In [ ]:
px.histogram(application_df, x="CODE_GENDER", color="TARGET",title= "Gender vs Target",barmode='group',text_auto=True)


In [ ]:
print(application_df["CODE_GENDER"].value_counts())

We have more Female applicants than Male and also we have more Females with loan defaulter cases. 

#### Age
In our dataset we have age given in number of days with. Convert it by dividing with 365 or 365.25(more accurate) and if dividing with 365 then later use the abs() function to make the age positive. 

In [ ]:
# application_df["Age"] = application_df.DAYS_BIRTH//(-365.25) 
# application_df[["Age"]].describe()

#or

application_df["Age"] = application_df.DAYS_BIRTH//(365.25) 
application_df["Age"] = abs(application_df["Age"])
application_df["Age"].describe()


We have got minimum age of applicant as 21 and maximum age of applicant that has applied is 70. 


In [ ]:
fig = plt.figure(figsize=(12,6))

ax1 = fig.add_subplot(1, 2, 1, title="Non-defaulter")
ax2 = fig.add_subplot(1, 2, 2, title="Defaulter")

sns.kdeplot(application_df[application_df["TARGET"] == 0]['Age'], ax=ax1)
sns.kdeplot(application_df[application_df["TARGET"] == 1]['Age'], ax=ax2)

plt.show()

Applicants in their 30's have highest cases of default and as the age goes 40 above then the default case has seen a decrease. 

#### Income and Annuity

In [ ]:
plt.figure(figsize=(10,2))
sns.boxplot(application_df['AMT_INCOME_TOTAL'])
plt.show()

In [ ]:
plt.figure(figsize=(10,2))
sns.boxplot(application_df['AMT_ANNUITY'])
plt.show()

In both box plots we can see that their are outliers, These outliers can be valid too but again this will impact the other records
Lets remove these outliers and then plot a graph. 

In [ ]:
application_df = application_df[application_df['AMT_ANNUITY'] < np.nanpercentile(application_df['AMT_ANNUITY'], 99)]
application_df = application_df[application_df['AMT_INCOME_TOTAL'] < np.nanpercentile(application_df['AMT_INCOME_TOTAL'], 99)]


In [ ]:
fig = plt.figure(figsize=(12,6))

ax1 = fig.add_subplot(1, 2, 1, title="Non-defaulter")
ax2 = fig.add_subplot(1, 2, 2, title="Defaulter")

sns.kdeplot(application_df[application_df["TARGET"] == 0]['AMT_INCOME_TOTAL'], ax=ax1)
sns.kdeplot(application_df[application_df["TARGET"] == 1]['AMT_INCOME_TOTAL'], ax=ax2)

plt.show()

In [ ]:
fig = plt.figure(figsize=(12,6))

ax1 = fig.add_subplot(1, 2, 1, title="Non-defaulter")
ax2 = fig.add_subplot(1, 2, 2, title="Defaulter")

sns.kdeplot(application_df[application_df["TARGET"] == 0]['AMT_ANNUITY'], ax=ax1)
sns.kdeplot(application_df[application_df["TARGET"] == 1]['AMT_ANNUITY'], ax=ax2)

plt.show()

Again, as we see that range between 100,000 and 150,000 as annual income have high chance of non repayment whereas the cases are less as annual income increases.

Amount Annuity(Monthy Installments). We see no differnece in the distribution for non defaulters and defaulters. 

#### Top Features with high correlation for Defaulter value

In [ ]:
default = application_df[application_df["TARGET"]==1]
default.drop(["SK_ID_CURR"],axis=1)

In [ ]:
defaulter_corr = default.corr()
round(defaulter_corr, 2)

corr_list = defaulter_corr.unstack()


In [ ]:
# Listing the correlations in pair sorted in descending order
corr_list.sort_values(ascending=False).drop_duplicates().head(11)